# 1° question
## Scrape the following Wikipedia page https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

### To create the dataframe I decided to use the wikipedia Python API. It's very easy to get the pandas dataframe.

In [5]:
!pip install wikipedia

  Stored in directory: /home/dsxuser/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


### To create the above dataframe:

- The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood    

- Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.    

- More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park.
  These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.     
  
- If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.     

- Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.     

- In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.   

In [6]:
import pandas as pd
import wikipedia as wp

html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")

canada_pc_df = pd.read_html(html)[0]
canada_pc_df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [7]:
df = canada_pc_df.rename(columns={'Postal Code':'PostalCode'})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [8]:
df = df[canada_pc_df.Neighborhood != 'Not assigned']
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.  

In [9]:
df_group = df.groupby(['PostalCode', 'Borough'], sort = False).agg( ','.join)
df_final = df_group.reset_index()
df_final.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [10]:
indexNum = df_final[df_final['Borough'] == 'Not assigned'].index
df_final.drop(indexNum, inplace = True)
df_final.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [11]:
print(df_final.shape)

(103, 3)


# 2° question
## Get the latitude and the longitude coordinates of each neighborhood

In [2]:
!pip install geocoder

     |████████████████████████████████| 102kB 9.3MB/s eta 0:00:01


#### Try to use geocoder without any success

In [23]:
# import geocoder # import geocoder

# def get_geocoder(postal_code_from_df):
#      # initialize your variable to None
#      lat_lng_coords = None
#      # loop until you get the coordinates
#      while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
#        lat_lng_coords = g.latlng
#      latitude = lat_lng_coords[0]
#      longitude = lat_lng_coords[1]
        
#      return latitude,longitude

# # df_final['Latitude'], df_final['Longitude'] = zip(*df_final['PostalCode'].apply(get_geocoder))

# # get_geocoder("M1B")

# geocoder.google('{}, Toronto, Ontario'.format("M1B"))

#### So I get geo coordinations from a file 

In [24]:
geo_tor_df = pd.read_csv('http://cocl.us/Geospatial_data')
geo_tor_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geo_tor_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [25]:
df_final_geo = pd.merge(geo_tor_df, df_final, on='PostalCode')[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
df_final_geo.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# 3° question
## Explore and cluster the neighborhoods in Toronto. 